# [STARTER] Exercise - Output structured Agent responses

In this exercise, you'll learn how to enhance your AI agent to provide structured outputs using Pydantic models. This will help ensure the agent's responses are consistent, validated, and easily usable in downstream applications.

## Challenge

You have an existing Agent class that can:
- Process user messages
- Use tools when needed
- Generate responses

Now you need to enhance it to:
- Define structured output formats using Pydantic
- Parse and validate responses
- Return data in a consistent JSON format



## Setup
First, let's import the necessary libraries:

In [1]:
from typing import List, Any, Annotated
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import json

from lib.messages import UserMessage, SystemMessage, ToolMessage
from lib.tooling import tool
from lib.llm import LLM
from lib.parsers import PydanticOutputParser, JsonOutputParser

## Defining Structured Output Models

Let's create a Pydantic model for a meeting summary with action items:


In [2]:
# DONE 1: Create the ActionItem Pydantic model
# Hint: Include fields for task, assignee, and due_date with appropriate annotations and descriptions

class ActionItem(BaseModel):
    task: str = Field(
        ...,
        description="Task description"
    )
    assignee: str = Field(
        ...,
        description="Person responsible for the task"
    )
    due_date: str = Field(
        ...,
        description="Due date for the task, typically in ISO format (YYYY-MM-DD)."
    )

In [3]:
# DONE 2: Create the MeetingSummary Pydantic model
# Hint: Include fields for title, date, participants, key_points, and action_items

class MeetingSummary(BaseModel):
    title: str = Field(
        ...,
        description="The title or subject of the meeting."
    )
    date: str = Field(
        ...,
        description="The date of the meeting, typically in ISO format (YYYY-MM-DD)."
    )
    participants: List[str] = Field(
        ...,
        description="A list of names of all attendees present in the meeting."
    )
    key_points: List[str] = Field(
        ...,
        description="A list of the main discussion points or decisions covered during the meeting."
    )
    action_items: List[ActionItem] = Field(
        ...,
        description="A list of follow-up tasks assigned during the meeting, each represented as an ActionItem."
    )

## Enhanced Agent Class

Now let's create an enhanced version of our Agent class that supports structured outputs:


In [8]:
class StructuredAgent:
    """An AI Agent that provides structured outputs"""
    
    def __init__(
        self,
        role: str = "Meeting Assistant",
        instructions: str = "Help summarize meetings and track action items",
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
        tools: List[Any] = None,
        output_model: BaseModel = None
    ):
        """Initialize the agent with its configuration
        
        Args:
            role: The agent's role/persona
            instructions: Basic instructions for the agent
            model: The LLM model to use
            temperature: Creativity parameter (0.0 = more deterministic)
            tools: List of tools the agent can use
            output_model: Pydantic model for structured output
        """
        # DONE 3: Initialize the agent
        # Hint:
        # - Store agent settings (role, instructions, output_model, etc.)
        # - Load environment variables
        # - Create an LLM instance with the provided configuration
        self.role = role
        self.instructions = instructions
        self.output_model = output_model

        load_dotenv()

        self.llm = LLM(
            model=model,
            temperature=temperature,
            tools=tools
        )


    def invoke(self, user_message: str) -> dict:
        """Process a user message and return a structured response
        
        Args:
            user_message: The user's input message
            
        Returns:
            A dictionary containing the structured response
        """
        # DONE 4: Implement the invoke method
        # Hint:
        # - Create messages list with SystemMessage
        # - Add UserMessage
        # - Get AI response with structured format if output_model exists
        # - Parse and return the response

        messages = [
            SystemMessage(content=f"""
Role: {self.role}
Instructions: {self.instructions}
            """),
            UserMessage(content=user_message)
        ]
        ai_message = self.llm.invoke(
            messages,
            self.output_model
        )
        messages.append(ai_message)

        while ai_message.tool_calls:
            for tool_call in ai_message.tool_calls:
                function_name = tool_call.function.name
                args = json.loads(tool_call.function.arguments)

                tool = self.llm.tools[function_name]
                if tool:
                    result = tool(**args)
                    messages.append(ToolMessage(
                        content=json.dumps(result),
                        tool_call_id = tool_call.id,
                        name = function_name
                    ))
                
            ai_message = self.llm.invoke(
                messages
            )
            messages.append(ai_message)

        if self.output_model:
            self.output_model.model_validate_json(ai_message.content)

        return json.loads(ai_message.content)


## Testing the Structured Agent

Let's test our enhanced agent with a meeting summary example:


In [9]:
# Create an agent instance with the MeetingSummary model
meeting_agent = StructuredAgent(
    role="Meeting Assistant",
    instructions="Summarize meetings and track action items in a structured format",
    output_model=MeetingSummary
)

In [11]:
meeting_transcript = """
Project Planning Meeting - March 15, 2024

Attendees: John, Sarah, Mike

Discussion:
- Reviewed Q1 project timeline
- Discussed resource allocation
- Identified potential risks

Next steps:
1. John will update the project plan by next Friday
2. Sarah needs to coordinate with the design team by Wednesday
3. Mike will prepare the risk assessment document by end of month
"""

In [12]:
summary = meeting_agent.invoke(meeting_transcript)
print(json.dumps(summary, indent=2))

{
  "title": "Project Planning Meeting",
  "date": "2024-03-15",
  "participants": [
    "John",
    "Sarah",
    "Mike"
  ],
  "key_points": [
    "Reviewed Q1 project timeline",
    "Discussed resource allocation",
    "Identified potential risks"
  ],
  "action_items": [
    {
      "task": "Update the project plan",
      "assignee": "John",
      "due_date": "2024-03-22"
    },
    {
      "task": "Coordinate with the design team",
      "assignee": "Sarah",
      "due_date": "2024-03-20"
    },
    {
      "task": "Prepare the risk assessment document",
      "assignee": "Mike",
      "due_date": "2024-03-31"
    }
  ]
}


## Validating the Output

Let's verify that our output matches our Pydantic model structure:


In [13]:
# Create a MeetingSummary instance from the output
validated_summary = MeetingSummary(**summary)

In [14]:
# Access structured data
print("Meeting Title:", validated_summary.title)
print("\nParticipants:")
for participant in validated_summary.participants:
    print(f"- {participant}")

print("\nAction Items:")
for item in validated_summary.action_items:
    print(f"- {item.task} (Assigned to: {item.assignee}, Due: {item.due_date})")

Meeting Title: Project Planning Meeting

Participants:
- John
- Sarah
- Mike

Action Items:
- Update the project plan (Assigned to: John, Due: 2024-03-22)
- Coordinate with the design team (Assigned to: Sarah, Due: 2024-03-20)
- Prepare the risk assessment document (Assigned to: Mike, Due: 2024-03-31)
